## Práctica Guiada Requests y Beautiful Soup

En esta Práctica Guiada usaremos BeautifulSoup para descargar la información de la primera división de fútbol de Argentina de la página de ESPN:

    http://www.espn.com.ar/futbol/posiciones/_/liga/arg.1

Debemos descargarla, extraerla, ordenarla y guardarla en un csv y/o un DataFrame para su uso. 
Primero importamos las librerias que necesitaremos.

In [0]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#Con esto podemos echar un vistazo a código HTML directamente en la Notebook
from IPython.display import HTML, display

Para chequear que no hay problema scrapeando esta información, revisamos el archivo "robots.txt"

In [0]:
robots = 'http://www.espn.com.ar/' + 'robots.txt'
print(requests.get(robots).text)

User-agent: *
Disallow: *print?id
Disallow: /format/
Disallow: /nfl/format/player/design09/dropdown
Disallow: /nba/format/player/design09/dropdown
Disallow: /golf/deportes/format/player/design09/dropdown
Disallow: /extra/mma/deportes/format/player/design09/dropdown
Disallow: /extra/mma/format/player/design09/dropdown
Disallow: /rpm/deportes/format/player/design09/dropdown
Disallow: /golf/format/player/design09/dropdown
Disallow: /mlb/format/player/design09/dropdown
Disallow: /futbol/copamundial2010/
Disallow: /members/v3_1/login
Disallow: /insider/
Disallow: /videohub/video/clip?id
Disallow: /futbolint/noticias/
Disallow: /futbolargentino/columnas/
Disallow: /blogs/null
Disallow: /personalization/v3_1/personalization
Disallow: /futbol/equipo/null
Disallow: /mundial2006/
Disallow: /webslices/




No aparece nuestra URL, entonces procedemos.

Hacemos un requests "GET" a la URL con la tabla, y vemos el contenido de la respuesta en la Notebook

In [0]:
url = 'http://www.espn.com.ar/futbol/posiciones/_/liga/arg.1'
resp = requests.get(url)

In [0]:
#Vemos el contenido renderizado
display(HTML(resp.text))

1Boca JuniorsCABJ,27,18,4,5,50,22,+28,58
2Godoy Cruz de MendozaGCM,27,17,5,5,45,24,+21,56
3San LorenzoSL,27,14,8,5,31,20,+11,50
4HuracánHUR,27,13,9,5,35,24,+11,48
5Talleres de CórdobaTDC,27,13,7,7,33,20,+13,46
6IndependienteIND,27,13,7,7,29,19,+10,46
7Racing ClubRAC,27,13,6,8,46,32,+14,45
8River PlateCARP,27,13,6,8,39,26,+13,45
9Defensa y JusticiaDYJ,27,13,5,9,41,34,+7,44
10Unión de Santa FeUSF,27,11,10,6,33,23,+10,43
11Colón de Santa FeCOL,27,11,8,8,32,22,+10,41


In [0]:
soup = BeautifulSoup(resp.text, 'html.parser')

Podemos acceder a "tags" directamente

In [0]:
soup.title

<title>Posiciones de la Superliga Argentina</title>

Para trabajar con el contenido vamos a usar algunos de los siguientes métodos que tiene este objeto como:

- .findAll()  /  .find_all
- .find()
- .get()
- .get_text()

In [0]:
soup.head.link.get('href')

'http://www.espn.com.ar/futbol/posiciones/_/liga/arg.1'

In [0]:
#Podemos usar find() para encontrar la tabla

raw_table = soup.find('table', {'class':'standings has-team-logos'})

In [0]:
type(raw_table)

bs4.element.Tag

Esto nos devuelve un objeto "Tag" definido por la librería BeautifulSoup. Podemos ver sus atributos usando .attrs

In [0]:
raw_table.attrs

{'cellpadding': '0',
 'cellspacing': '0',
 'class': ['standings', 'has-team-logos'],
 'data-behavior': 'responsive_table',
 'data-fix-cols': '1',
 'data-mobile-force-responsive': 'true',
 'data-set-cell-heights': 'false',
 'data-text-contract': 'Contract table',
 'data-text-expand': 'Expand table'}

Dentro del atributo "children" encontramos todos los descendientes de ese nodo. En el caso de nuestra tabla, los descendientes son tags "tr", que corresponden a cada fila. Existe otro atributo llamado "descendants", que a diferencia del primero es recursivo. En este caso solo necesitamos los nódos directamente próximos, así que usamos el primer método.

Iteramos todas las filas y formamos una matriz con los datos, para luego cargarlos a un DataFrame.

In [0]:
rows = []
for row in raw_table.children:
    rows.append(row.get_text(separator= ','))
rows

['',
 '',
 '',
 '',
 '2017/18 Superliga,PJ,G,E,P,PF,GC,DIF,PTS',
 '1,Boca Juniors,CABJ,27,18,4,5,50,22,+28,58',
 '2,Godoy Cruz de Mendoza,GCM,27,17,5,5,45,24,+21,56',
 '3,San Lorenzo,SL,27,14,8,5,31,20,+11,50',
 '4,Huracán,HUR,27,13,9,5,35,24,+11,48',
 '5,Talleres de Córdoba,TDC,27,13,7,7,33,20,+13,46',
 '6,Independiente,IND,27,13,7,7,29,19,+10,46',
 '7,Racing Club,RAC,27,13,6,8,46,32,+14,45',
 '8,River Plate,CARP,27,13,6,8,39,26,+13,45',
 '9,Defensa y Justicia,DYJ,27,13,5,9,41,34,+7,44',
 '10,Unión de Santa Fe,USF,27,11,10,6,33,23,+10,43',
 '11,Colón de Santa Fe,COL,27,11,8,8,32,22,+10,41',
 '12,Argentinos Juniors,ARGJ,27,12,5,10,36,30,+6,41',
 '13,Belgrano de Córdoba,BEL,27,10,10,7,29,28,+1,40',
 '14,Vélez Sarsfield,VEL,27,10,8,9,31,32,-1,38',
 '15,Atlético Tucumán,CAT,27,8,12,7,29,26,+3,36',
 '16,Estudiantes La Plata,EST,27,10,6,11,25,26,-1,36',
 '17,Banfield,BAN,27,9,8,10,27,24,+3,35',
 '18,San Martín de San Juan,SMSJ,27,9,6,12,30,36,-6,33',
 '19,Patronato,CAP,27,8,9,10,26,32,-6,33

In [0]:
#Separamos las columnas, y desechamos las filas vacias
table = [row.split(',') for row in rows if len(row) > 1]

In [0]:
#Corregimos la tabla

table[0] =  ['Index', 'Name', 'Abbr'] + table[0][1:]
print(*table, sep = '\n')

['Index', 'Name', 'Abbr', 'PJ', 'G', 'E', 'P', 'PF', 'GC', 'DIF', 'PTS']
['1', 'Boca Juniors', 'CABJ', '27', '18', '4', '5', '50', '22', '+28', '58']
['2', 'Godoy Cruz de Mendoza', 'GCM', '27', '17', '5', '5', '45', '24', '+21', '56']
['3', 'San Lorenzo', 'SL', '27', '14', '8', '5', '31', '20', '+11', '50']
['4', 'Huracán', 'HUR', '27', '13', '9', '5', '35', '24', '+11', '48']
['5', 'Talleres de Córdoba', 'TDC', '27', '13', '7', '7', '33', '20', '+13', '46']
['6', 'Independiente', 'IND', '27', '13', '7', '7', '29', '19', '+10', '46']
['7', 'Racing Club', 'RAC', '27', '13', '6', '8', '46', '32', '+14', '45']
['8', 'River Plate', 'CARP', '27', '13', '6', '8', '39', '26', '+13', '45']
['9', 'Defensa y Justicia', 'DYJ', '27', '13', '5', '9', '41', '34', '+7', '44']
['10', 'Unión de Santa Fe', 'USF', '27', '11', '10', '6', '33', '23', '+10', '43']
['11', 'Colón de Santa Fe', 'COL', '27', '11', '8', '8', '32', '22', '+10', '41']
['12', 'Argentinos Juniors', 'ARGJ', '27', '12', '5', '10', '36

Para guardar el archivo a un csv usando Python, usamos:

In [0]:
filename = 'tabla.csv'
with open(filename, 'w') as out:
    out.write('\n'.join([','.join(row) for row in table]))

In [0]:
df = pd.DataFrame(table[1:], columns= table[0])

In [0]:
df.columns

Index(['Index', 'Name', 'Abbr', 'PJ', 'G', 'E', 'P', 'PF', 'GC', 'DIF', 'PTS'], dtype='object')

In [0]:
df.set_index('Index', inplace=True)
df.head()

,Name,Abbr,PJ,G,E,P,PF,GC,DIF,PTS
Index,,,,,,,,,,
1,Boca Juniors,CABJ,27,18,4,5,50,22,+28,58
2,Godoy Cruz de Mendoza,GCM,27,17,5,5,45,24,+21,56
3,San Lorenzo,SL,27,14,8,5,31,20,+11,50
4,Huracán,HUR,27,13,9,5,35,24,+11,48
5,Talleres de Córdoba,TDC,27,13,7,7,33,20,+13,46


Por último, podemos guardar el archivo usando pandas con:

In [0]:
filename = 'tabla.csv'
df.to_csv(filename)